# 類別變數特徵工程

## 作業程式碼
本作業將請學員完成以下要求：
1. 請至 Kaggle 平台找尋欲探索的資料集，進行本次作業。
2. 請挑選適合的兩個類別變數，分別使用兩種不同撰寫方法執行 Label Encoding
3. 請挑選適合的兩個類別變數，分別使用兩種不同撰寫方法執行 One-Hot Encoding 
4. 請挑選適合的兩個類別變數，分別從三種方法中使用兩種不同撰寫方法執行 Ordinal Encoding
5. 請挑選適合的兩個類別變數，撰寫並執行 Frequency Encoding
6. 請挑選適合的兩個類別變數，撰寫並執行Feature Combination

In [2]:
import numpy as np
import pandas as pd

## 輸入資料

In [5]:
# 輸入資料
df = pd.read_csv("disaster_train.csv") # 此行需要填入資料路徑
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
# 為學習方便，在此先移除遺失值
df = df.dropna()

## Label Encoding

本次介紹以下兩種方法可進行 Label Encoding
1. 使用 sklearn 套件中 LabelEncoder 函數
2. 使用 Dictionary 資料型態的功能

舉例：將 Destination 進行 Label Encoding

In [7]:
# 方法一：使用 sklearn 套件中的 LabelEncoder 函數
from sklearn.preprocessing import LabelEncoder

df_le = df.copy()
le = LabelEncoder()
df_le["Sex_LE"] = le.fit_transform(df_le["Sex"])
df_le[["Sex", "Sex_LE"]].head(3)

,Sex,Sex_LE
1,female,0
3,female,0
6,male,1


In [8]:
df["Embarked"].value_counts()

Embarked
S    116
C     65
Q      2
Name: count, dtype: int64

In [10]:
# 方法二：使用 Dictionary 資料型態的功能
embarked_mapping = {"S": 0, "C":1, "Q":2}
df_le["Embarked_LE"] = df_le["Embarked"].map(embarked_mapping)
df_le["Embarked_LE"].value_counts()

Embarked_LE
0    116
1     65
2      2
Name: count, dtype: int64

## One-Hot Encoding

本次介紹以下兩種方法進行 One-Hot Encoding
1. 使用 sklearn 套件中的 OneHotEncoder 函數
2. 使用 Dictionary 資料型態的功能

舉例：將 Destination 變數進行 One-Hot Encoding

In [29]:
df["Pclass"].value_counts()

Pclass
1    158
2     15
3     10
Name: count, dtype: int64

In [30]:
# 方法一：使用 sklearn 套件中的 OneHotEncoder 函數
from sklearn.preprocessing import OneHotEncoder

df_ohe = df.copy()
ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
pclass_ohe = ohe.fit_transform(df_ohe[["Pclass"]]) # output a matrix
df_pclass_ohe = pd.DataFrame(pclass_ohe, columns=ohe.get_feature_names_out(["Pclass"]))
df_pclass_ohe.head(3)


,Pclass_1,Pclass_2,Pclass_3
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0


In [31]:
df_ohe = pd.concat([df_ohe.reset_index(drop=True), df_pclass_ohe], axis=1)
df_ohe.head(1)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Pclass_1,Pclass_2,Pclass_3
0,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1.0,0.0,0.0


> 注意：使用 OneHotEncoder 轉換時，資料要先把維度轉換成二維才能轉換喔

In [32]:
# 方法二：使用 Dictionary 資料型態的功能
df_embarked_ohe = pd.get_dummies(
    df_ohe["Embarked"],
    prefix="Embarked"
)

df_embarked_ohe.head(3)

,Embarked_C,Embarked_Q,Embarked_S
0,True,False,False
1,False,False,True
2,False,False,True


In [33]:
df_ohe = pd.concat([df_ohe, df_embarked_ohe], axis=1)

In [35]:
df_ohe[["Embarked","Embarked_C","Embarked_Q","Embarked_S"]].head(3)

,Embarked,Embarked_C,Embarked_Q,Embarked_S
0,C,True,False,False
1,S,False,False,True
2,S,False,False,True


## Ordinal Encoding

本次介紹兩種方法進行 Ordinal Encoding
1. 使用 sklearn 套件中的 LabelEncoder 函數且要自定義類別順序（比較不推薦）
2. 使用 sklearn 套件中的 OrdinalEncoder 函數且要自定義類別順序（比較推薦）
3. 使用 Dictionary 資料型態的功能


In [38]:
df["Pclass"].value_counts()

Pclass
1    158
2     15
3     10
Name: count, dtype: int64

In [47]:
# 方法一：使用 sklearn 套件中的 LabelEncoder 函數且要自定義類別順序
from sklearn.preprocessing import LabelEncoder

df_le = df.copy()

df_le["Pclass_custom"] = df_le["Pclass"].map({
    1: "high",
    2: "middle",
    3: "low"
})

le = LabelEncoder()
df_le["Pclass_label"] = le.fit_transform(df_le["Pclass_custom"])

df_le[["Pclass", "Pclass_custom", "Pclass_label"]].head(3)

,Pclass,Pclass_custom,Pclass_label
1,1,high,0
3,1,high,0
6,1,high,0


In [45]:
# 方法二：使用 sklearn 中的 OrdinalEncoder 函數
from sklearn.preprocessing import OrdinalEncoder

df_ord2 = df.copy()

# 將數值類別轉為字串
df_ord2["Pclass_str"] = df_ord2["Pclass"].astype(str)

oe = OrdinalEncoder(categories=[["3", "2", "1"]])

df_ord2["Pclass_ordinal"] = oe.fit_transform(df_ord2[["Pclass_str"]])

df_ord2[["Pclass", "Pclass_ordinal"]].head(3)

,Pclass,Pclass_ordinal
1,1,2.0
3,1,2.0
6,1,2.0


In [40]:
# 方法三：使用 Dictionary 資料型態的功能
df_ord3 = df.copy()
pclass_order = {
    3: 0,
    2: 1,
    1: 2
}

df_ord3["Pclass_ordinal"] = df_ord3["Pclass"].map(pclass_order)
df_ord3[["Pclass", "Pclass_ordinal"]].head(3)

,Pclass,Pclass_ordinal
1,1,2
3,1,2
6,1,2


## Frequency Encoding

本次主要使用 Dictionary 資料型態的功能實現 Frequency Encoding，步驟如下：
1. 先計算各類別的數量
2. 將計算結果轉換成 Dictionary 資料型態
3. 進行類別轉換

### quantity

In [ ]:
df_freq = df.copy()
freq_map = df_freq["Embarked"].value_counts()
df_freq["Embarked_freq"] = df_freq["Embarked"].map(freq_map)
df_freq[["Embarked", "Embarked_freq"]].head()

,Embarked,Embarked_freq
1,C,65
3,S,116
6,S,116
10,S,116
11,S,116


### ratio

In [53]:
freq_ratio = df_freq["Embarked"].value_counts(normalize=True)
df_freq['Embarked_freq_ratio'] = df_freq['Embarked'].map(freq_ratio)
df_freq[["Embarked", "Embarked_freq_ratio"]].head()

,Embarked,Embarked_freq_ratio
1,C,0.355191
3,S,0.633880
6,S,0.633880
10,S,0.633880
11,S,0.633880


## Feature Combination

本次將介紹使用 Dictionary 資料型態的功能進行類別特徵的特徵合併，其步驟如下：
1. 建構合併規則，並以 Dictionary 資料型態呈現
2. 將類別變數作轉換

In [ ]:
# Step1. 把 類別變數一 與 類別變數二 的類別取出來
sex_categories = df['Sex'].dropna().unique()
pclass_categories = sorted(df['Pclass'].dropna().unique())

# Step2. 建立一個二類別變數 Dict
combine_dict = {}
for sex in sex_categories:
    for pclass in pclass_categories:
        combine_dict[(sex, pclass)] = f"{sex}_{pclass}"

combine_dict

{('female', 1): 'female_1',
 ('female', 2): 'female_2',
 ('female', 3): 'female_3',
 ('male', 1): 'male_1',
 ('male', 2): 'male_2',
 ('male', 3): 'male_3'}

In [58]:
# Step3. 在資料表中產生新特徵
df_fc = df.copy()
df_fc["Sex_Pclass"] = df_fc.apply(
    lambda row: combine_dict.get((row['Sex'], row['Pclass']), 'Unknown'),
    axis=1
)

df_fc[["Sex", "Pclass", "Sex_Pclass"]].head(15)

,Sex,Pclass,Sex_Pclass
1,female,1,female_1
3,female,1,female_1
6,male,1,male_1
10,female,3,female_3
11,female,1,female_1
21,male,2,male_2
23,male,1,male_1
27,male,1,male_1
52,female,1,female_1
54,male,1,male_1
